# take2

**GOAL**: Keep It Stupid Simple = KISS.

**Dependencies:**
 - [pocketknives](https://github.com/zjwilliams20/pocketknives)

**References:**
 1. [ilqgames/python](https://github.com/HJReachability/ilqgames/blob/master/python)

In [ ]:
from time import perf_counter as pc
import warnings

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

from decentralized import split_agents, plot_solve
import decentralized as dec
import pocketknives

π = np.pi
g = 9.80665

In [ ]:
%matplotlib ipympl
plt.rcParams['axes.grid'] = False

In [ ]:
plt.rc('font',size=12)
plt.rc('font',family='palatino')
plt.rc('axes',labelsize=14)

In [ ]:
# fig = plt.figure(figsize=(3.7, 3.7))
fig = plt.figure()

## single-agent problem

Show equations of motion, which have the form

$$\dot{x} = f(x, u, p)$$

for states

$$
x = \begin{bmatrix} o_x & o_y & o_z & \psi & \theta & \phi & v_x & v_y & v_z & w_x & w_y & w_z \end{bmatrix},
$$

inputs
$$
u = \begin{bmatrix} \tau_x & \tau_y & \tau_z & f_z \end{bmatrix},
$$

and parameters
$$
p = \begin{bmatrix} m & J_x & J_y & J_z & g \end{bmatrix}.
$$

In [ ]:
# Single-agent problem for a 12D quadcopter (w/ analytical diff.)
dt = 0.1
N = 40
n_d = 3

x = np.array([2, 2, 0.5 , 0.05, 0, 0, 0.0, 0.0, 0, 0, 0, 0], dtype=float)
xf = np.zeros((12, 1), dtype=float).T

dynamics = dec.QuadcopterDynamics12D(dt)

# Q = np.diag([1., 1., 1., 0, 0, 0, 0, 0, 0, 0, 0, 0])
Q = np.diag([1., 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Qf = 100 * np.eye(Q.shape[0])
R = np.eye(4)
cost = dec.ReferenceCost(xf, Q, R, Qf)

prob = dec.ilqrProblem(dynamics, cost)
ilqr = dec.ilqrSolver(prob, N)

# with warnings.catch_warnings():
    # warnings.filterwarnings("error")
X, U, J = ilqr.solve(x)

plt.clf()
plot_solve(X, J, xf, n_d=n_d)
plt.draw()

In [ ]:
# Single-agent problem for a 6D quadcopter (w/ analytical diff.)
dt = 0.1
N = 40
n_d = 3

x = np.array([2, 2, 0.5 , 0, 0, 0], dtype=float)
xf = np.zeros((6, 1), dtype=float).T

dynamics = dec.QuadcopterDynamics6D(dt)

# Q = np.diag([1., 1., 1., 0, 0, 0, 0, 0, 0, 0, 0, 0])
Q = np.diag([1., 1., 1., 1., 1., 1.])
Qf = 100 * np.eye(Q.shape[0])
R = np.diag([0, 1, 1])
cost = dec.ReferenceCost(xf, Q, R, Qf)

prob = dec.ilqrProblem(dynamics, cost)
ilqr = dec.ilqrSolver(prob, N)

# with warnings.catch_warnings():
    # warnings.filterwarnings("error")
X, U, J = ilqr.solve(x)

plt.clf()
plot_solve(X, J, xf, n_d=n_d)
plt.draw()

## Human Agent Alone

In [ ]:
dt = 0.1
N = 40
n_d = 3

x = np.array([2, 2, 1.75 , 0, 0, 0], dtype=float)
xf = np.zeros((6, 1), dtype=float).T
xf[:,2] = 1.75
dynamics = dec.HumanDynamics6D(dt)


Q = np.diag([1., 1., 1., 1., 1., 1.])
Qf = 100 * np.eye(Q.shape[0])
R = np.eye(3)
cost = dec.ReferenceCost(xf, Q, R, Qf)

prob = dec.ilqrProblem(dynamics, cost)
ilqr = dec.ilqrSolver(prob, N)

X, U, J = ilqr.solve(x)

In [ ]:
plt.clf()
plot_solve(X, J, xf, n_d=n_d)
plt.draw()

## multi-agent problem

### initializing the scene
Here, we define some initial positions within some distance of each other, and then rotate them about the origin by some random angle to hopefully create some interesting scenarios.

In [ ]:
def paper_setup():
    """Hardcoded example with reasonable consistency eyeballed from 
       Potential-iLQR paper
    """
    x0 = np.array([[0.5, 1.5, 0, 0.1,
                    2.5, 1.5, 0, π,
                    1.5, 1.3, 0, π/2]], 
                     dtype=float).T
    xf = np.array([[2.5, 1.5, 0, 0, 
                        0.5, 1.5, 0, π, 
                        1.5, 2.2, 0, π/2]]).T
    return x0, xf

def dec_test_setup():
    x0, xf = paper_setup()
    x0_other = np.array([[5, 5, 0, 0,
                          6, 6, 0, π/4]]).T
    xf_other = np.array([[6, 4, 0, -π/2,
                              4, 6, 0, π/4]]).T
    
    x0 = np.concatenate([x0, x0_other]) 
    xf = np.concatenate([xf, xf_other])    
    
    return x0, xf

def paper_setup_3_quads():
    x0 = np.array([[0.5, 1.5, 1, 0, 0, 0,
                    2.5, 1.5, 1, 0, 0, 0,
                    1.5, 1.3, 1, 0, 0, 0]], 
                     dtype=float).T
    xf = np.array([[2.5, 1.5, 1, 0, 0, 0, 
                    0.5, 1.5, 1, 0, 0, 0, 
                    1.5, 2.2, 1, 0, 0, 0]]).T
    x0[dec.pos_mask([6]*3, 3)] += 0.01*np.random.randn(9, 1)
    xf[dec.pos_mask([6]*3, 3)] += 0.01*np.random.randn(9, 1)
    return x0, xf

In [ ]:
def paper_example_unicycles():
    """5 agents with crossing, N = 60, R = 2I"""
    x0 = np.array([[-3.894,  1.453,  0.   ,  0.   ,  0.873,  4.82 ,  0.   ,  0.   ,
        -3.372, -3.415,  0.   ,  0.   ,  3.502,  4.681,  0.   ,  0.   ,
         4.592,  0.605,  0.   ,  0.   ]]).T
    xf = np.array([[ 4.877,  2.181,  0.   ,  0.   ,  1.928, -0.136,  0.   ,  0.   ,
        -1.879,  3.688,  0.   ,  0.   , -2.026, -0.522,  0.   ,  0.   ,
         4.043, -3.767,  0.   ,  0.   ]]).T
    return x0, xf

In [ ]:
def paper_example_quads():
    """10 quadcopters, N = 60"""
    x0 = np.array([[ 5.915, -7.175,  1.903,  0.   ,  0.   ,  0.   ,  1.838,  2.391,
         5.71 ,  0.   ,  0.   ,  0.   ,  3.706,  3.317, -8.283,  0.   ,
         0.   ,  0.   ,  6.694,  7.111,  6.155,  0.   ,  0.   ,  0.   ,
        -9.541,  0.501,  2.488,  0.   ,  0.   ,  0.   , -4.286, -7.567,
        -7.29 ,  0.   ,  0.   ,  0.   ,  3.574, -4.309, -1.497,  0.   ,
         0.   ,  0.   ,  0.417, -2.895,  0.594,  0.   ,  0.   ,  0.   ,
        -6.272, -6.414, -1.265,  0.   ,  0.   ,  0.   ,  7.108, -9.521,
        -5.234,  0.   ,  0.   ,  0.   ]]).T
    xf = np.array([[ 0.586,  6.324,  8.746,  0.   ,  0.   ,  0.   , -5.768, -5.674,
         9.156,  0.   ,  0.   ,  0.   ,  1.384,  6.137,  3.046,  0.   ,
         0.   ,  0.   , -9.182,  1.458, -5.108,  0.   ,  0.   ,  0.   ,
        -4.171, -3.339, -6.949,  0.   ,  0.   ,  0.   , -9.142, -1.331,
         7.007,  0.   ,  0.   ,  0.   , -2.239, -7.678,  3.992,  0.   ,
         0.   ,  0.   ,  0.704, -4.572,  9.133,  0.   ,  0.   ,  0.   ,
         4.2  , -8.159,  4.172,  0.   ,  0.   ,  0.   ,  7.384, -1.938,
        -5.644,  0.   ,  0.   ,  0.   ]]).T

    return x0, xf

## quadcopter w/ humans

In [ ]:
n_states = 6
n_controls = 3
n_agents = 3

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents
n_dims = [2, 3, 3]

dt = 0.1
N = 20
radius = 1.0

x0 = np.array([
    [-0.2, 2, 0, -π/2, 0, 0], 
    [-2, 1.6, 1.75, -0.01, -0.01, 0.0],
    [0.5, 1.0, 1.73, 0.01, -0.01, 0.0]
]).flatten()
xf = np.array([
    [0.4, -2, 0, -π/2, 0, 0], 
    [1, -1.9, 1.75, 0, 0, 0],
    [1, 1.9, 1.73, 0, 0, 0]
]).flatten() 

Q = np.diag([1, 1, 1, 1, 1, 1])
R = np.diag([1, 1, 1])
Qf = 1e3 * np.eye(n_states)

models = [dec.HumanDynamics6D, dec.QuadcopterDynamics6D, dec.QuadcopterDynamics6D]
ids = [100 + i for i in range(n_agents)]
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_, model in zip(ids, models)])

goal_costs = [
    dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_)
    for xf_i, id_, x_dim, u_dim in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, u_dims
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
solver = dec.ilqrSolver(problem, N)

In [ ]:
U0 = np.zeros((N, n_agents*n_controls))
X, U, J, _ = dec.solve_centralized(solver, x0, U0, ids, True)

In [ ]:
plt.clf()
plot_solve(X, J, xf, x_dims, n_d=3)
plt.title('One-shot decentralized trajectory with Human and Quad')
plt.draw()

In [ ]:
U0 = np.zeros((N, n_agents*n_controls))
X0 = np.tile(x0, (N+1, 1))
X_dec, U_dec, J_dec, _ = dec.solve_decentralized(problem, X0, U0, radius, verbose=True)

In [ ]:
plt.clf()
plot_solve(X_dec, J_dec, xf, x_dims, n_d=3)
plt.title('One-shot decentralized trajectory with Human and Quad')
plt.draw()

In [ ]:
plt.clf()
plt.plot(dec.compute_pairwise_distance(X, x_dims, n_dims[1]))
plt.hlines(radius, *plt.xlim(), 'r', ls='--')
plt.title('Distance between human and quad over entire horizon')
plt.ylabel('Pairwise distance (m)',fontsize=13)
plt.draw()

In [ ]:
%%time
n_d = 3
step_size = 1
N = 30

# Start in hover condition.
# U0 = np.zeros((N, n_agents*n_controls))
U0 = np.c_[
    np.zeros((N, 3)),
    np.tile([g, 0, 0], (N, 2))
]

radius = 0.5
X_rhc, U_rhc, J_rhc = dec.solve_rhc(
    problem, x0, N, radius,
    centralized=False,
    n_d=n_d,
    step_size=step_size, 
    dist_converge=0.1,
    verbose=True,
    t_kill=step_size*dt,
    t_diverge=None
)

In [ ]:
plt.clf()
plot_solve(X_rhc, J_rhc, xf, x_dims, n_d=3)
plt.title('RHC with human and quad')
plt.draw()

In [ ]:
plt.clf()
plt.plot(dec.compute_pairwise_distance(X, x_dims, n_dims[1]))
plt.hlines(radius, *plt.xlim(), 'r', ls='--')
plt.title('Distance between human and quad over entire horizon')
plt.ylabel('Pairwise distance (m)',fontsize=13)
plt.draw()

To be consistent between simulations, we normalize for the scale of the setup by computing the *energy*, or the sum of distances from the origin of the points. This should be the same for all runs.

### uniform agents

In [ ]:
def paper_setup_3_quads():
    x0 = np.array([[0.5, 1.5, 1, 0.5, 0.0, 0,
                    2.5, 1.5, 1, -0.5, 0.0, 0,
                    1.5, 1.3, 1, 0, 0.2, 0]], 
                     dtype=float).T
    xf = np.array([[2.5, 1.5, 1, 0, 0, 0, 
                    0.5, 1.5, 1, 0, 0, 0, 
                    1.5, 2.2, 1, 0, 0, 0]]).T
    x0[dec.pos_mask([6]*3, 3)] += 0.1*np.random.randn(9, 1)
    xf[dec.pos_mask([6]*3, 3)] += 0.1*np.random.randn(9, 1)
    return x0, xf

In [ ]:
n_states = 6
n_controls = 3
n_agents = 3
x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents
n_dims = [3] * n_agents

# ENERGY = 20.0
n_d = n_dims[0]

# x0, xf = paper_setup()
# x0, xf = car_setup()
# x0, xf = bike_setup()
# x0, xf = double_int_setup()
# x0, xf = dec_test_setup()
# x0, xf = dec.random_setup(
#     n_agents, n_states, 
#     is_rotation=False, 
#     rel_dist=2.0, 
#     var=n_agents/2, 
#     n_d=2, 
#     random=True
# )

x0, xf = paper_setup_3_quads()
# x0, xf = paper_example_unicycles()

x_dims = [n_states] * n_agents
u_dims = [n_controls] * n_agents

# x0 = dec.normalize_energy(x0, x_dims, ENERGY)
# xf = dec.normalize_energy(xf, x_dims, ENERGY)
# x0 = dec.perturb_state(x0, x_dims)

plt.clf()
plt.gca().set_aspect('equal')
X = np.dstack([
    x0.reshape(n_agents, n_states), 
    xf.reshape(n_agents, n_states)
]).swapaxes(1,2)
for i, Xi in enumerate(X):
    plt.annotate('', Xi[1,:2], Xi[0,:2], 
                 arrowprops=dict(facecolor=plt.cm.tab20.colors[i]))
pocketknives.set_bounds(X.reshape(-1, n_states), zoom=0.2)
plt.draw()

In [ ]:
dt = 0.05
N = 30

tol = 1e-6
ids = [100 + i for i in range(n_agents)]

# model = dec.DoubleIntDynamics4D
# model = dec.UnicycleDynamics4D
# model = dec.BikeDynamics5D
model = dec.QuadcopterDynamics6D
dynamics = dec.MultiDynamicalModel([model(dt, id_) for id_ in ids])

# Q = np.eye(4)
# R = np.eye(2)
Q = np.diag([1, 1, 1, 5, 5, 5])
R = np.diag([0, 1, 1])
Qf = 1e3 * np.eye(n_states)
radius = 0.5

goal_costs = [
    dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_)
    for xf_i, id_, x_dim, u_dim in zip(
        dec.split_agents_gen(xf, x_dims), ids, x_dims, u_dims
    )
]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
goal_costs = [dec.ReferenceCost(xf_i, Q.copy(), R.copy(), Qf.copy(), id_) 
              for xf_i, id_ in zip(split_agents(xf.T, x_dims), ids)]
prox_cost = dec.ProximityCost(x_dims, radius, n_dims)
game_cost = dec.GameCost(goal_costs, prox_cost)

problem = dec.ilqrProblem(dynamics, game_cost)
solver = dec.ilqrSolver(problem, N)

# U0 = None

In [ ]:
%%time
U0 = np.tile([1.5*g, 0, 0.0], (N, n_agents))
U0[:, ~dec.pos_mask([3]*3, 1)] = 1e-3*np.random.randn(N, n_agents*(n_controls-1))
X, U, J = solver.solve(x0, U0, tol=tol, t_kill=None)

In [ ]:
# fname = "10-quads"
# fname = "5-unicycles"
fname = "_"

In [ ]:
plt.clf()
plot_solve(X, J, xf.T, x_dims, True, n_d)

ax = plt.gca()
# ax.set_title("5 Unicycle Agents")
# ax.set_xlabel("$x$")
# ax.set_ylabel("$y$")
# ax.set_zlabel("$z$")
ax.legend(plt.gca().get_children()[1:3], ["Start Position", "Goal Position"])
# pocketknives.ticks_off()
ax.set_title('')
# ax.legend(plt.gca().get_children()[1:3], ["Start Position", "Goal Position"], loc=(-0.2, 0.75))
# ax.yaxis.set_major_locator(plt.MultipleLocator(3))
# plt.savefig(f"/home/iconlab/Downloads/{fname}.png")
# plt.gcf().set_facecolor("#EAEAF2")
plt.gca().set_zlim(0, 1.5)
plt.draw()


In [ ]:
plt.clf()
plt.plot(dec.compute_pairwise_distance(X, x_dims, n_d))
plt.hlines(radius, 0, N, 'r', ls='--', label="$d_{prox}$")
plt.title("Inter-Agent Distances")
plt.xlabel("Time Steps")
plt.ylabel("Distance")
plt.legend()
plt.grid(False)
plt.draw()
# plt.savefig(f"/home/iconlab/Downloads/{fname}-distances.png")

In [ ]:
costs = []
for xi, ui in zip(X, U):
    costs.append(game_cost(xi, ui).item())
plt.clf()
plt.plot(costs)
plt.draw()

## decentralized multi-agent

In [ ]:
graph = dec.define_inter_graph_threshold(X, 4*radius, x_dims, ids)
graph

In [ ]:
nx.draw_networkx?

In [ ]:
plt.clf()
dec.plot_interaction_graph(graph)
plt.title("One-Shot Interaction Graph")

In [ ]:
plt.grid(False)
plt.savefig("/home/iconlab/Downloads/graph.png")

In [ ]:
# %%time
X0 = np.tile(x0.T, (N,1))
U0 = np.zeros((N, n_controls*n_agents))
X_dec1, U_dec1, J_dec, _ = dec.solve_decentralized(problem, X0, U0, radius)

In [ ]:
plt.clf()
plot_solve(X_dec1, J_dec, xf, x_dims, n_d)
plt.gca().set_zlim(-1, 1)
plt.draw()

In [ ]:
plt.clf()
plt.plot(dec.compute_pairwise_distance(X_dec, x_dims, n_d))
plt.hlines(radius, *plt.xlim(), 'r', ls='--')
plt.draw()

## With receding horizon:

In [ ]:
%%time
step_size = 3
U0 = np.zeros((N, dynamics.n_u))
N = 5
radius = 0.25
X, U, J = dec.solve_rhc(
    prob, x0, N, radius,
    centralized=True,
    n_d=n_d,
    step_size=step_size, 
    dist_converge=0.1,
    verbose=True,
    t_kill=step_size*dt,
    t_diverge=N*dt
)

In [ ]:
np.isclose(x0,X[0]) #shouldn't x0 be exactly the same as X[0]??

In [ ]:
plt.figure(dpi=100)
plot_solve(X, J, xf, x_dims, n_d=3)
plt.savefig('/mnt/c/Users/Randy666/Documents/projects/ICON_lab/decentralized/media/Centralized_15_drones.png')

In [ ]:
%%time
#Decentralized without multiprocessing:
step_size = 3
U0 = np.zeros((N, dynamics.n_u))
N = 50
radius = 0.25
X, U, J = dec.solve_rhc(
    prob, x0, N, radius,
    centralized=False,
    n_d=n_d,
    step_size=step_size, 
    dist_converge=0.1,
    verbose=True,
    t_kill=step_size*dt,
    t_diverge=N*dt
)

In [ ]:
plt.figure()
plot_solve(X, J, xf, x_dims, n_d=3)
plt.savefig('/mnt/c/Users/Randy666/Documents/projects/ICON_lab/decentralized/media/decentralized_15_drones.png')

## animation

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(5.0, 3.7))

In [ ]:
def setup(axes, X, xf, x_dims, radius):
    
    ax1, ax2 = axes
    n_agents = len(x_dims)
    handles1 = []
    for i in range(n_agents):
        handles1.append((
            ax1.plot(0, c=plt.cm.tab20.colors[2*i], marker='o', markersize=4)[0],
            ax1.add_artist(plt.Circle(np.nan, radius, color='k', fill=True, alpha=0.3, lw=2)),
        ))
        
    for xg in split_agents(xf, x_dims):
        ax1.scatter(xg[0,0], xg[0,1], c='r', marker='x', zorder=10)

    X_cat = np.vstack(split_agents(X, x_dims))
    pocketknives.set_bounds(X_cat, axes[0], zoom=0.15)
    plt.draw()
    
    handles2 = []
    n_pairs = pocketknives.nchoosek(n_agents, 2)
    for i in range(n_pairs):
        handles2.append(ax2.plot(0, c=plt.cm.tab20.colors[2*i])[0])
    ax2.hlines(radius, 0, X.shape[0], 'r', ls='--')
    ax2.set_ylim(0.0, distances.max())
        
    return handles1, handles2
    
def animate(t, handles1, handles2, X, x_dims, xf, distances):
    """Animate the solution into a gif"""
    
    for (i, xi), hi in zip(enumerate(split_agents(X, x_dims)), handles1):
        hi[0].set_xdata(xi[:t,0])
        hi[0].set_ydata(xi[:t,1])
        hi[1].set_center(xi[t-1,:2])
        
    for i, hi in enumerate(handles2):
        hi.set_xdata(range(t))
        hi.set_ydata(distances[:t, i])
    
    plt.draw()
    

In [ ]:
distances = dec.compute_pairwise_distance(X, x_dims).T
handles1, handles2 = setup(axes, X, xf.flatten(), x_dims, radius)
animate(10, handles1, handles2, X, x_dims, xf, distances)
plt.draw()

In [ ]:
handles = setup(axes, X, xf.flatten(), x_dims, radius)
anim = FuncAnimation(
    plt.gcf(), 
    animate, 
    frames=N+1, 
    fargs=(*handles, X, x_dims, xf, distances))
anim.save('traj.gif', fps=5)

## old

In [ ]:
# Old RHC for plotting...
rhc = dec.RecedingHorizonController(x0, ilqr, step_size=5)
U0 = np.zeros((ilqr.N, ilqr.n_u))
for Xi, Ui, Ji in rhc.solve(U0, J_converge=101.0, tol=tol, verbose=False):
    plt.cla()
    plot_solve(Xi, Ji, xf.T, x_dims)
    fig.canvas.draw()

In [ ]:
# Manual decentralized RHC for plotting...
step_size = 5
Xi = np.tile(x0.T, (N,1))
Ui = np.zeros((N, dynamics.n_u))
X_dec = np.zeros((0, dynamics.n_x))
U_dec = np.zeros((0, dynamics.n_u))
Ji = np.inf
J_converge = 100.0

while Ji >= J_converge:
    Xi, Ui, Ji = dec.solve_decentralized(prob, Xi, Ui, radius)
    
    X_dec = np.r_[X_dec, Xi[:step_size]]
    U_dec = np.r_[U_dec, Ui[:step_size]]
    
    # Seed the next solve by staying at the last visited state.
    Xi = np.r_[Xi[step_size:], np.tile(Xi[-1], (step_size, 1))]
    Ui = np.r_[Ui[step_size:], np.zeros((step_size, dynamics.n_u))]
    
    plt.cla()
    plot_solve(Xi[:step_size], Ji, xf.T, x_dims)
    fig.canvas.draw()